---

_You are currently looking at **version 1.0** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-text-mining/resources/d9pwm) course resource._

---

# Assignment 4 - Document Similarity & Topic Modelling

## Part 1 - Document Similarity

For the first part of this assignment, you will complete the functions `doc_to_synsets` and `similarity_score` which will be used by `document_path_similarity` to find the path similarity between two documents.

The following functions are provided:
* **`convert_tag:`** converts the tag given by `nltk.pos_tag` to a tag used by `wordnet.synsets`. You will need to use this function in `doc_to_synsets`.
* **`document_path_similarity:`** computes the symmetrical path similarity between two documents by finding the synsets in each document using `doc_to_synsets`, then computing similarities using `similarity_score`.

You will need to finish writing the following functions:
* **`doc_to_synsets:`** returns a list of synsets in document. This function should first tokenize and part of speech tag the document using `nltk.word_tokenize` and `nltk.pos_tag`. Then it should find each tokens corresponding synset using `wn.synsets(token, wordnet_tag)`. The first synset match should be used. If there is no match, that token is skipped.
* **`similarity_score:`** returns the normalized similarity score of a list of synsets (s1) onto a second list of synsets (s2). For each synset in s1, find the synset in s2 with the largest similarity value. Sum all of the largest similarity values together and normalize this value by dividing it by the number of largest similarity values found. Be careful with data types, which should be floats. Missing values should be ignored.

Once `doc_to_synsets` and `similarity_score` have been completed, submit to the autograder which will run `test_document_path_similarity` to test that these functions are running correctly. 

*Do not modify the functions `convert_tag`, `document_path_similarity`, and `test_document_path_similarity`.*

In [1]:
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import re

In [2]:
'''a = [None]*5
b = list(filter(None, a))
if not b:
    print (b)
#is 0 = the same word? no can't it's 1/(n+1)'''

"a = [None]*5\nb = list(filter(None, a))\nif not b:\n    print (b)\n#is 0 = the same word? no can't it's 1/(n+1)"

In [3]:
'''#VERSION 1.0
#I HATE THIS
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd


def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
    """
    Returns a list of synsets in document.

    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.

    Args:
        doc: string to be converted

    Returns:
        list of synsets

    Example:
        doc_to_synsets('Fish are nvqjp friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """
    

    # Your Code Here
    
    
    #remove everything punctuation excluding spaces
    #edit: removing numbers as well but maybe that's NOT good
    x = re.sub(r'([\W]|[\d])+', ' ', doc)
    x = x.lower()
    #tokenize list and lower
    tokens = list(nltk.word_tokenize(x))
    mask = [True] * len(tokens)
    for i in range(len(tokens)):
        if len(wn.synsets(tokens[i])) == 0:
            mask[i] = False
        else:
            mask[i] = True
    tokens = list(np.array(tokens)[np.array(mask)])
    #do i need this if i use sysnet(s) = answer: yes because or ARE = VERB NOT NOUN, OMG_MF
    #get pos_tags to remove None
    pos_token = nltk.pos_tag(tokens)
    z = map(convert_tag, pos_token)
    df = pd.DataFrame(pos_token)
    df.columns = ['word', 'pos']
    df['pos'] = list(map(convert_tag,df['pos']))
    df.dropna(inplace=True)
    
    #lemmatize to the correct word usage
    lemma_list = [None]*len(df)
    for i in range(len(lemma_list)):
        lemma_list[i] = lemmatizer.lemmatize(df.iloc[i,0],df.iloc[i,1])
    df['word'] = lemma_list
    df['combo'] = df['word']+'.'+df['pos']+'.01'

    prep_tokens = list(df['combo'])
    
    syns = [None]*len(prep_tokens)
    for t in range(len(syns)):
        #try:
        syns[t] = wn.synset(prep_tokens[t])
        #except Exception:
            #continue
    syns = [x for x in syns if x is not None]
    
    return syns


def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """
    
    
    x1 = s1
    x2 = s2

    syn_scores = [None]*len(x1)
    for i in range(len(x1)):
        #print(i)
        temp_scores = [None]*len(x2)
        for j in range(len(x2)):
            #print (i,j)
            temp_scores[j] = x1[i].path_similarity(x2[j])
            #print(x1[i],x2[j],":",temp_scores[j])])
            #print(temp_scores[j])
        #print(temp_scores)
        temp_scores = [x for x in temp_scores if x != None]
        if len(temp_scores) == 0:
            syn_scores[i] = None
        else:
            syn_scores[i] = max(temp_scores)
        #temp_scores = filter(0, temp_scores)
        #syn_scores[i] = max(temp_scores)
    syn_scores = list(filter(None, syn_scores))
    return np.mean(syn_scores)


def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2'''

'#VERSION 1.0\n#I HATE THIS\nimport numpy as np\nimport nltk\nfrom nltk.corpus import wordnet as wn\nimport pandas as pd\n\n\ndef convert_tag(tag):\n    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""\n    \n    tag_dict = {\'N\': \'n\', \'J\': \'a\', \'R\': \'r\', \'V\': \'v\'}\n    try:\n        return tag_dict[tag[0]]\n    except KeyError:\n        return None\n\n\ndef doc_to_synsets(doc):\n    """\n    Returns a list of synsets in document.\n\n    Tokenizes and tags the words in the document doc.\n    Then finds the first synset for each word/tag combination.\n    If a synset is not found for that combination it is skipped.\n\n    Args:\n        doc: string to be converted\n\n    Returns:\n        list of synsets\n\n    Example:\n        doc_to_synsets(\'Fish are nvqjp friends.\')\n        Out: [Synset(\'fish.n.01\'), Synset(\'be.v.01\'), Synset(\'friend.n.01\')]\n    """\n    \n\n    # Your Code Here\n    \n    \n    #remove everything punctuation ex

In [4]:
import numpy as np
import nltk
from nltk.corpus import wordnet as wn
import pandas as pd

#VERSION 1.5

def convert_tag(tag):
    """Convert the tag given by nltk.pos_tag to the tag used by wordnet.synsets"""
    
    tag_dict = {'N': 'n', 'J': 'a', 'R': 'r', 'V': 'v'}
    try:
        return tag_dict[tag[0]]
    except KeyError:
        return None


def doc_to_synsets(doc):
    """
    Returns a list of synsets in document.

    Tokenizes and tags the words in the document doc.
    Then finds the first synset for each word/tag combination.
    If a synset is not found for that combination it is skipped.

    Args:
        doc: string to be converted

    Returns:
        list of synsets

    Example:
        doc_to_synsets('Fish are nvqjp friends.')
        Out: [Synset('fish.n.01'), Synset('be.v.01'), Synset('friend.n.01')]
    """
    

    # Your Code Here
    
    x = doc
    x = re.sub(r'([\W]|[\d])+', ' ', x)
    x = x.lower()

    #version 1.5
    tokens = list(nltk.word_tokenize(x))
    #insert lemma
    pos_token = nltk.pos_tag(tokens)
    pos_list = [x[1] for x in pos_token]
    pos_list = list(map(convert_tag,pos_list))
    token_pos = list(zip(tokens, pos_list))
    prep_tokens = [x for x in token_pos if x[1] != None]
    new_tokens = [x[0] for x in prep_tokens]
    
    #removed for 8:13 PM trial
    #lemma here
    #lemma_list = [None]*len(prep_tokens)
    #for i in range(len(lemma_list)):
    #    lemma_list[i] = lemmatizer.lemmatize(prep_tokens[i][0],prep_tokens[i][1])
    #final_pos = [x[1] for x in prep_tokens]
    #final_tokens = list(zip(lemma_list, final_pos))
    #lemma there, dilemma of the lemma
    
    syns = [None]*len(new_tokens)
    for t in range(len(syns)):
        if len(wn.synsets(new_tokens[t])) == 0:
            syns[t] = None
        else:
            syns[t] = wn.synsets(new_tokens[t])[0]
            #syns[t] = wn.synsets(final_tokens[t][0], final_tokens[t][1])[0]
        #except Exception:
        #    continue
    syns = [x for x in syns if x is not None]
      
    return syns


def similarity_score(s1, s2):
    """
    Calculate the normalized similarity score of s1 onto s2

    For each synset in s1, finds the synset in s2 with the largest similarity value.
    Sum of all of the largest similarity values and normalize this value by dividing it by the
    number of largest similarity values found.

    Args:
        s1, s2: list of synsets from doc_to_synsets

    Returns:
        normalized similarity score of s1 onto s2

    Example:
        synsets1 = doc_to_synsets('I like cats')
        synsets2 = doc_to_synsets('I like dogs')
        similarity_score(synsets1, synsets2)
        Out: 0.73333333333333339
    """
    
    
    x1 = s1
    x2 = s2

    syn_scores = [None]*len(x1)
    for i in range(len(x1)):
        #print(i)
        temp_scores = [None]*len(x2)
        for j in range(len(x2)):
            #print (i,j)
            temp_scores[j] = x1[i].path_similarity(x2[j])
            #print(x1[i],x2[j],":",temp_scores[j])])
            #print(temp_scores[j])
        #print(temp_scores)
        temp_scores = [x for x in temp_scores if x != None]
        if len(temp_scores) == 0:
            syn_scores[i] = None
        else:
            syn_scores[i] = max(temp_scores)
        #temp_scores = filter(0, temp_scores)
        #syn_scores[i] = max(temp_scores)
    syn_scores = list(filter(None, syn_scores))
    return np.mean(syn_scores)


def document_path_similarity(doc1, doc2):
    """Finds the symmetrical similarity between doc1 and doc2"""

    synsets1 = doc_to_synsets(doc1)
    synsets2 = doc_to_synsets(doc2)

    return (similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2

In [5]:
doc1 = 'This is a function to test document_path_similarity.'
doc2 = 'Use this function to see if your code in doc_to_synsets \
and similarity_score is correct!'
document_path_similarity(doc1, doc2)

0.55706145706145715

In [6]:
'''x = "The card is issued by Mexico's consulates to its citizens living abroad and shows the date of birth, a current photograph and the address of the cardholder."
x = re.sub(r'([\W]|[\d])+', ' ', x)
x = x.lower()
x'''

'x = "The card is issued by Mexico\'s consulates to its citizens living abroad and shows the date of birth, a current photograph and the address of the cardholder."\nx = re.sub(r\'([\\W]|[\\d])+\', \' \', x)\nx = x.lower()\nx'

In [7]:
x = 'Ms Stewart, the chief executive, was not expected to attend.'
x = re.sub(r'([\W]|[\d])+', ' ', x)
x = x.lower()

#version 1.5
tokens = list(nltk.word_tokenize(x))
#insert lemma
pos_token = nltk.pos_tag(tokens)
pos_list = [x[1] for x in pos_token]
pos_list = list(map(convert_tag,pos_list))
token_pos = list(zip(tokens, pos_list))
prep_tokens = [x for x in token_pos if x[1] != None]
new_tokens = [x[0] for x in prep_tokens]
#lemma here
#lemma_list = [None]*len(prep_tokens)
#for i in range(len(lemma_list)):
#    lemma_list[i] = lemmatizer.lemmatize(prep_tokens[i][0],prep_tokens[i][1])
#final_pos = [x[1] for x in prep_tokens]
#final_tokens = list(zip(lemma_list, prep_tokens[1]))
#lemma there, dilemma of the lemma

'''
#version 1.0
tokens2 = list(nltk.word_tokenize(x))
mask = [True] * len(tokens)
for i in range(len(tokens)):
    if len(wn.synsets(tokens[i])) == 0:
        mask[i] = False
    else:
        mask[i] = True
tokens = list(np.array(tokens)[np.array(mask)])
      
    #do i need this if i use sysnet(s)
    #get pos_tags to remove None
pos_token = nltk.pos_tag(tokens)
        #z = map(convert_tag, pos_token)
df = pd.DataFrame(pos_token)
df.columns = ['word', 'pos']

df['pos'] = list(map(convert_tag,df['pos']))
df.dropna(inplace=True)
   
    #lemmatize to the correct word usage
lemma_list = [None]*len(df)
for i in range(len(lemma_list)):
    lemma_list[i] = lemmatizer.lemmatize(df.iloc[i,0],df.iloc[i,1])
df['word'] = lemma_list
df['combo'] = df['word']+'.'+df['pos']+'.01'

#re_tuple
pos_token_clean = [tuple(x) for x in df.values]
pos_token_clean[0]

prep_tokens = list(df['combo'])
wn.synsets('ms','a')
#df'''
new_tokens

['ms', 'stewart', 'chief', 'executive', 'was', 'not', 'expected', 'attend']

In [8]:
#important maybe lemma if needed?
'''lemma_list = [None]*len(prep_tokens)
for i in range(len(lemma_list)):
    lemma_list[i] = lemmatizer.lemmatize(prep_tokens[i][0],prep_tokens[i][1])
lemma_list'''

'lemma_list = [None]*len(prep_tokens)\nfor i in range(len(lemma_list)):\n    lemma_list[i] = lemmatizer.lemmatize(prep_tokens[i][0],prep_tokens[i][1])\nlemma_list'

In [9]:
wn.synsets('lazy')


[Synset('lazy.s.01'), Synset('faineant.s.01')]

In [10]:
'''syns = [None]*len(final_tokens)
for t in range(len(syns)):
    if len(wn.synsets(final_tokens[t][0], final_tokens[t][1])) == 0:
        syns[t] = None
    else:
        syns[t] = wn.synsets(final_tokens[t][0], final_tokens[t][1])[0]
    #except Exception:
    #    continue
syns = [x for x in syns if x is not None]
syns'''

'syns = [None]*len(final_tokens)\nfor t in range(len(syns)):\n    if len(wn.synsets(final_tokens[t][0], final_tokens[t][1])) == 0:\n        syns[t] = None\n    else:\n        syns[t] = wn.synsets(final_tokens[t][0], final_tokens[t][1])[0]\n    #except Exception:\n    #    continue\nsyns = [x for x in syns if x is not None]\nsyns'

In [11]:
'''#https://stackoverflow.com/questions/27591621/nltk-convert-tokenized-sentence-to-synset-format
from nltk.corpus import wordnet as wn

def penn_to_wn(tag):
    if tag.startswith('J'):
        return wn.ADJ
    elif tag.startswith('N'):
        return wn.NOUN
    elif tag.startswith('R'):
        return wn.ADV
    elif tag.startswith('V'):
        return wn.VERB
    return None'''

"#https://stackoverflow.com/questions/27591621/nltk-convert-tokenized-sentence-to-synset-format\nfrom nltk.corpus import wordnet as wn\n\ndef penn_to_wn(tag):\n    if tag.startswith('J'):\n        return wn.ADJ\n    elif tag.startswith('N'):\n        return wn.NOUN\n    elif tag.startswith('R'):\n        return wn.ADV\n    elif tag.startswith('V'):\n        return wn.VERB\n    return None"

In [12]:
'''from nltk.stem import WordNetLemmatizer
from nltk import pos_tag, word_tokenize

sentence = "'Fish are nvqjp friends.'"
tagged = pos_tag(word_tokenize(sentence))

synsets = [None]*len(sentence)
lemmatzr = WordNetLemmatizer()

for token in tagged:

    try:
        wn_tag = penn_to_wn(token[1])
        if not wn_tag:
            continue
            lemma = lemmatzr.lemmatize(token[0], pos=wn_tag)
            synsets.append(wn.synsets(lemma, pos=wn_tag)[0])
    except IndexError:
        pass

print (synsets)'''

'from nltk.stem import WordNetLemmatizer\nfrom nltk import pos_tag, word_tokenize\n\nsentence = "\'Fish are nvqjp friends.\'"\ntagged = pos_tag(word_tokenize(sentence))\n\nsynsets = [None]*len(sentence)\nlemmatzr = WordNetLemmatizer()\n\nfor token in tagged:\n\n    try:\n        wn_tag = penn_to_wn(token[1])\n        if not wn_tag:\n            continue\n            lemma = lemmatzr.lemmatize(token[0], pos=wn_tag)\n            synsets.append(wn.synsets(lemma, pos=wn_tag)[0])\n    except IndexError:\n        pass\n\nprint (synsets)'

In [13]:
#doc_to_synsets()
#synsets1 = doc_to_synsets('I like cats')
#synsets2 = doc_to_synsets('I like dogs')

'''x1 = doc_to_synsets('I like cats')
x2 = doc_to_synsets('I like dogs')
mnlength = min(len(x1),len(x2))
mxlength = max(len(x1),len(x2))

syn_scores = [0]*mnlength
for i in range(mnlength):
    temp_scores = [0]*mxlength
    for j in range(mxlength):
        temp_scores[j] = wn.path_similarity(x1[i], x2[j])
        #print(temp_scores[j])
    temp_scores = filter(None, temp_scores)
    syn_scores[i] = max(temp_scores)

type(np.mean(syn_scores))'''

"x1 = doc_to_synsets('I like cats')\nx2 = doc_to_synsets('I like dogs')\nmnlength = min(len(x1),len(x2))\nmxlength = max(len(x1),len(x2))\n\nsyn_scores = [0]*mnlength\nfor i in range(mnlength):\n    temp_scores = [0]*mxlength\n    for j in range(mxlength):\n        temp_scores[j] = wn.path_similarity(x1[i], x2[j])\n        #print(temp_scores[j])\n    temp_scores = filter(None, temp_scores)\n    syn_scores[i] = max(temp_scores)\n\ntype(np.mean(syn_scores))"

### test_document_path_similarity

Use this function to check if doc_to_synsets and similarity_score are correct.

*This function should return the similarity score as a float.*

In [14]:
def test_document_path_similarity():
    doc1 = 'This is a function to test document_path_similarity.'
    doc2 = 'Use this function to see if your code in doc_to_synsets \
    and similarity_score is correct!'
    return document_path_similarity(doc1, doc2)

In [15]:
test_document_path_similarity()

0.55706145706145715

<br>
___
`paraphrases` is a DataFrame which contains the following columns: `Quality`, `D1`, and `D2`.

`Quality` is an indicator variable which indicates if the two documents `D1` and `D2` are paraphrases of one another (1 for paraphrase, 0 for not paraphrase).

In [16]:
# Use this dataframe for questions most_similar_docs and label_accuracy
paraphrases = pd.read_csv('paraphrases.csv')
paraphrases.head(5)
#len(paraphrases)
paraphrases.iloc[0,1]

'Ms Stewart, the chief executive, was not expected to attend.'

___

### most_similar_docs

Using `document_path_similarity`, find the pair of documents in paraphrases which has the maximum similarity score.

*This function should return a tuple `(D1, D2, similarity_score)`*

In [17]:
'''a = [0.1111111111111111, 1.0, 0.25, 0.14285714285714285, 1.0]
np.mean(a) == sum(a)/len(a)'''

'a = [0.1111111111111111, 1.0, 0.25, 0.14285714285714285, 1.0]\nnp.mean(a) == sum(a)/len(a)'

In [18]:
'''deer = wn.synset('deer.n.01')
elk = wn.synset('elk.n.01')
deer.path_similarity(elk)'''

"deer = wn.synset('deer.n.01')\nelk = wn.synset('elk.n.01')\ndeer.path_similarity(elk)"

In [19]:
'''doc1 = 'This is a function to test document_path_similarity.'
doc2 = 'Use this function to see if your code in doc_to_synsets \
and similarity_score is correct!'
#D1 = paraphrases.iloc[2,1]
#D2 = paraphrases.iloc[2,2]
#DS1 = doc_to_synsets(D1)
#DS2 = doc_to_synsets(D2)
DC1 = doc_to_synsets(doc1)
DC2 = doc_to_synsets(doc2)
#print(DS1[0],DS2[0])
#DS1[0].path_similarity(DS2[0])'''

"doc1 = 'This is a function to test document_path_similarity.'\ndoc2 = 'Use this function to see if your code in doc_to_synsets and similarity_score is correct!'\n#D1 = paraphrases.iloc[2,1]\n#D2 = paraphrases.iloc[2,2]\n#DS1 = doc_to_synsets(D1)\n#DS2 = doc_to_synsets(D2)\nDC1 = doc_to_synsets(doc1)\nDC2 = doc_to_synsets(doc2)\n#print(DS1[0],DS2[0])\n#DS1[0].path_similarity(DS2[0])"

In [20]:
'''x2 = DC1
x1 = DC2

syn_scores = [None]*len(x1)
for i in range(len(x1)):
    #print(i)
    temp_scores = [None]*len(x2)
    for j in range(len(x2)):
        #print (i,j)
        temp_scores[j] = x1[i].path_similarity(x2[j])
        #print(x1[i],x2[j],":",temp_scores[j])
        #print(temp_scores[j])
    #print(temp_scores)
    temp_scores = [x for x in temp_scores if x != None]
    if len(temp_scores) == 0:
        syn_scores[i] = None
    else:
        syn_scores[i] = max(temp_scores)
    print (syn_scores)
    #temp_scores = filter(0, temp_scores)
    #syn_scores[i] = max(temp_scores)
syn_scores = list(filter(None, syn_scores))
print ("FINAL:",syn_scores)
np.mean(syn_scores)'''

'x2 = DC1\nx1 = DC2\n\nsyn_scores = [None]*len(x1)\nfor i in range(len(x1)):\n    #print(i)\n    temp_scores = [None]*len(x2)\n    for j in range(len(x2)):\n        #print (i,j)\n        temp_scores[j] = x1[i].path_similarity(x2[j])\n        #print(x1[i],x2[j],":",temp_scores[j])\n        #print(temp_scores[j])\n    #print(temp_scores)\n    temp_scores = [x for x in temp_scores if x != None]\n    if len(temp_scores) == 0:\n        syn_scores[i] = None\n    else:\n        syn_scores[i] = max(temp_scores)\n    print (syn_scores)\n    #temp_scores = filter(0, temp_scores)\n    #syn_scores[i] = max(temp_scores)\nsyn_scores = list(filter(None, syn_scores))\nprint ("FINAL:",syn_scores)\nnp.mean(syn_scores)'

In [21]:
'''synsets1 = doc_to_synsets(doc1)
synsets2 = doc_to_synsets(doc2)
(similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2'''

'synsets1 = doc_to_synsets(doc1)\nsynsets2 = doc_to_synsets(doc2)\n(similarity_score(synsets1, synsets2) + similarity_score(synsets2, synsets1)) / 2'

In [22]:
'''pp_list = [None]*len(paraphrases)
for i in  range(len(paraphrases)):
    #print(i)
    sim_score = document_path_similarity(paraphrases.iloc[i,1], paraphrases.iloc[i,2])
    pp_list[i] = (paraphrases.iloc[i,1], paraphrases.iloc[i,2], sim_score)

pp_df = pd.DataFrame(pp_list)
pp_df.columns = ['D1', 'D2','sim_score']
pp_df['sim_score'].argmax()
#ans = (pp_df.iloc[4,0],pp_df.iloc[4,1],pp_df.iloc[4,2])
#ans
pp_df'''

"pp_list = [None]*len(paraphrases)\nfor i in  range(len(paraphrases)):\n    #print(i)\n    sim_score = document_path_similarity(paraphrases.iloc[i,1], paraphrases.iloc[i,2])\n    pp_list[i] = (paraphrases.iloc[i,1], paraphrases.iloc[i,2], sim_score)\n\npp_df = pd.DataFrame(pp_list)\npp_df.columns = ['D1', 'D2','sim_score']\npp_df['sim_score'].argmax()\n#ans = (pp_df.iloc[4,0],pp_df.iloc[4,1],pp_df.iloc[4,2])\n#ans\npp_df"

In [23]:
def most_similar_docs():
    
    pp_list = [None]*len(paraphrases)
    for i in  range(len(paraphrases)):
        #print(i)
        sim_score = document_path_similarity(paraphrases.iloc[i,1], paraphrases.iloc[i,2])
        pp_list[i] = (paraphrases.iloc[i,1], paraphrases.iloc[i,2], sim_score)

    pp_df = pd.DataFrame(pp_list)
    pp_df.columns = ['D1', 'D2','sim_score']
    x = pp_df['sim_score'].argmax()
    ans = (pp_df.iloc[x,0],pp_df.iloc[x,1],pp_df.iloc[x,2])
   
    return ans

In [24]:
most_similar_docs()

('"Indeed, Iran should be put on notice that efforts to try to remake Iraq in their image will be aggressively put down," he said.',
 '"Iran should be on notice that attempts to remake Iraq in Iran\'s image will be aggressively put down," he said.\n',
 0.96825396825396826)

### label_accuracy

Provide labels for the twenty pairs of documents by computing the similarity for each pair using `document_path_similarity`. Let the classifier rule be that if the score is greater than 0.75, label is paraphrase (1), else label is not paraphrase (0). Report accuracy of the classifier using scikit-learn's accuracy_score.

*This function should return a float.*

In [25]:
pp_list = [None]*len(paraphrases)
for i in  range(len(paraphrases)):
    #print(i)
    sim_score = document_path_similarity(paraphrases.iloc[i,1], paraphrases.iloc[i,2])
    pp_list[i] = (paraphrases.iloc[i,1], paraphrases.iloc[i,2], sim_score)

pp_df = pd.DataFrame(pp_list)
pp_df.columns = ['D1', 'D2','sim_score']
predicted = pp_df['sim_score']
predicted = [1 if x >.75 else 0 for x in predicted]
#[ unicode(x.strip()) if x is not None else '' for x in row ]
y_pred = predicted
y_true = paraphrases['Quality']
from sklearn.metrics import accuracy_score
accuracy_score(y_true, y_pred)
print(y_pred)

[1, 1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1]


In [26]:
def label_accuracy():
    from sklearn.metrics import accuracy_score

    pp_list = [None]*len(paraphrases)
    for i in  range(len(paraphrases)):
        #print(i)
        sim_score = document_path_similarity(paraphrases.iloc[i,1], paraphrases.iloc[i,2])
        pp_list[i] = (paraphrases.iloc[i,1], paraphrases.iloc[i,2], sim_score)

    pp_df = pd.DataFrame(pp_list)
    pp_df.columns = ['D1', 'D2','sim_score']
    predicted = pp_df['sim_score']
    predicted = [1 if x >.75 else 0 for x in predicted]
    #[ unicode(x.strip()) if x is not None else '' for x in row ]
    y_pred = predicted
    y_true = paraphrases['Quality']
    omg = accuracy_score(y_true, y_pred)
    
    return omg

In [27]:
label_accuracy()

0.75

## Part 2 - Topic Modelling

For the second part of this assignment, you will use Gensim's LDA (Latent Dirichlet Allocation) model to model topics in `newsgroup_data`. You will first need to finish the code in the cell below by using gensim.models.ldamodel.LdaModel constructor to estimate LDA model parameters on the corpus, and save to the variable `ldamodel`. Extract 10 topics using `corpus` and `id_map`, and with `passes=25` and `random_state=34`.

In [28]:
import pickle
import gensim
from sklearn.feature_extraction.text import CountVectorizer

# Load the list of documents
with open('newsgroups', 'rb') as f:
    newsgroup_data = pickle.load(f)

# Use CountVectorizor to find three letter tokens, remove stop_words, 
# remove tokens that don't appear in at least 20 documents,
# remove tokens that appear in more than 20% of the documents
vect = CountVectorizer(min_df=20, max_df=0.2, stop_words='english', 
                       token_pattern='(?u)\\b\\w\\w\\w+\\b')
# Fit and transform
X = vect.fit_transform(newsgroup_data)

# Convert sparse matrix to gensim corpus.
corpus = gensim.matutils.Sparse2Corpus(X, documents_columns=False)

# Mapping from word IDs to words (To be used in LdaModel's id2word parameter)
id_map = dict((v, k) for k, v in vect.vocabulary_.items())


In [29]:
# Use the gensim.models.ldamodel.LdaModel constructor to estimate 
# LDA model parameters on the corpus, and save to the variable `ldamodel`

# Your code here:
ldamodel = gensim.models.ldamodel.LdaModel(corpus, id2word=id_map, num_topics=10, passes=25, random_state=34)

In [30]:
#from gensim import print_topics


### lda_topics

Using `ldamodel`, find a list of the 10 topics and the most significant 10 words in each topic. This should be structured as a list of 10 tuples where each tuple takes on the form:

`(9, '0.068*"space" + 0.036*"nasa" + 0.021*"science" + 0.020*"edu" + 0.019*"data" + 0.017*"shuttle" + 0.015*"launch" + 0.015*"available" + 0.014*"center" + 0.014*"sci"')`

for example.

*This function should return a list of tuples.*

In [31]:
def lda_topics():
    
    # Your Code Here
    
    return ldamodel.print_topics(num_topics = 10, num_words = 10)

In [32]:
lda_topics()

[(0,
  '0.056*"edu" + 0.043*"com" + 0.033*"thanks" + 0.022*"mail" + 0.021*"know" + 0.020*"does" + 0.014*"info" + 0.012*"monitor" + 0.010*"looking" + 0.010*"don"'),
 (1,
  '0.024*"ground" + 0.018*"current" + 0.018*"just" + 0.013*"want" + 0.013*"use" + 0.011*"using" + 0.011*"used" + 0.010*"power" + 0.010*"speed" + 0.010*"output"'),
 (2,
  '0.061*"drive" + 0.042*"disk" + 0.033*"scsi" + 0.030*"drives" + 0.028*"hard" + 0.028*"controller" + 0.027*"card" + 0.020*"rom" + 0.018*"floppy" + 0.017*"bus"'),
 (3,
  '0.023*"time" + 0.015*"atheism" + 0.014*"list" + 0.013*"left" + 0.012*"alt" + 0.012*"faq" + 0.012*"probably" + 0.011*"know" + 0.011*"send" + 0.010*"months"'),
 (4,
  '0.025*"car" + 0.016*"just" + 0.014*"don" + 0.014*"bike" + 0.012*"good" + 0.011*"new" + 0.011*"think" + 0.010*"year" + 0.010*"cars" + 0.010*"time"'),
 (5,
  '0.030*"game" + 0.027*"team" + 0.023*"year" + 0.017*"games" + 0.016*"play" + 0.012*"season" + 0.012*"players" + 0.012*"win" + 0.011*"hockey" + 0.011*"good"'),
 (6,
  '0.0

### topic_distribution

For the new document `new_doc`, find the topic distribution. Remember to use vect.transform on the the new doc, and Sparse2Corpus to convert the sparse matrix to gensim corpus.

*This function should return a list of tuples, where each tuple is `(#topic, probability)`*

In [33]:
new_doc = ["\n\nIt's my understanding that the freezing will start to occur because \
of the\ngrowing distance of Pluto and Charon from the Sun, due to it's\nelliptical orbit. \
It is not due to shadowing effects. \n\n\nPluto can shadow Charon, and vice-versa.\n\nGeorge \
Krumins\n-- "]

In [58]:
yvect = CountVectorizer(stop_words='english', token_pattern='(?u)\\b\\w\\w\\w+\\b')
Y = yvect.fit_transform(new_doc)
ycorpus = gensim.matutils.Sparse2Corpus(Y, documents_columns=False)
doc_lda = ldamodel[ycorpus]
print(list(ldamodel.get_document_topics(ycorpus, minimum_probability=0)) == list(doc_lda))
list(doc_lda)[0]

False


[(0, 0.15937947493053653),
 (1, 0.0047637338927391066),
 (2, 0.0047625021838596463),
 (3, 0.0047621723486367144),
 (4, 0.0047626662877813825),
 (5, 0.004764893956661311),
 (6, 0.80251613979454839),
 (7, 0.0047633068918455012),
 (8, 0.0047626206888141251),
 (9, 0.0047624890245773158)]

In [53]:
def topic_distribution():
    
    yvect = CountVectorizer(stop_words='english', 
                           token_pattern='(?u)\\b\\w\\w\\w+\\b')
    Y = yvect.fit_transform(new_doc)
    ycorpus = gensim.matutils.Sparse2Corpus(Y, documents_columns=False)
    doc_lda = ldamodel[ycorpus]
    list(ldamodel.get_document_topics(ycorpus))
    #doc_lda = ldamodel[ycorpus]
    
    return list(ldamodel.get_document_topics(ycorpus, minimum_probability=0))[0]

In [54]:
topic_distribution()

[(0, 0.15940167658429105),
 (1, 0.0047637337083774878),
 (2, 0.004762502043672498),
 (3, 0.0047621723065000192),
 (4, 0.004762666227396733),
 (5, 0.0047646872598276551),
 (6, 0.80249414549839104),
 (7, 0.0047633068104143078),
 (8, 0.0047626205485930932),
 (9, 0.0047624890125359948)]

### topic_names

From the list of the following given topics, assign topic names to the topics you found. If none of these names best matches the topics you found, create a new 1-3 word "title" for the topic.

Topics: Health, Science, Automobiles, Politics, Government, Travel, Computers & IT, Sports, Business, Society & Lifestyle, Religion, Education.

*This function should return a list of 10 strings.*

In [55]:
def topic_names():
    
    # Your Code Here
    
    return ['Education', 'Electricity', 'Computer Science', 'Religion', 'Travel', 'Sports', 'Health', 'Thoughts', 'Computers & IT', 'Space']

In [38]:
len(topic_names())

10